In [1]:
from queue import Queue
import numpy as np

In [12]:
N = 'N'
S = 'S'
E = 'E'
W = 'W'

# use graph search 


# this will probably need to be an environment variable
def expert_navigation_policy_set(desc, loc):
    (loc_r, loc_c) = loc
    num_rows = len(desc)
    num_cols = len(desc[0])
    max_row = num_rows - 1
    max_col = num_cols - 1
    spt = [[[[], np.inf] for _ in range(num_cols)] for _ in range(num_rows)]
    
    
    spt[loc_r][loc_c][0].append(-1) # after arriving at dest, no action should be taken
    spt[loc_r][loc_c][1] = 0
    
    q = Queue()
    q.put([loc, 0])
    
    while not q.empty():
        [(row, col), cur_dist] = q.get() # current location we are exploring the neighbors of
        for action in ['N', 'E', 'S', 'W']: 
            if action == 'S':
                (next_r, next_c) = (min(row + 1, max_row), col)
                if spt[next_r][next_c][1] >= cur_dist + 1 and desc[next_r][next_c]!= 3:
                    if spt[next_r][next_c][1] > cur_dist + 1:  
                        spt[next_r][next_c][1] = cur_dist + 1
                        q.put([(next_r, next_c), cur_dist + 1])
                    spt[next_r][next_c][0].append(N) # add to optimal actions list
                                     
            elif action == 'N':
                (next_r, next_c) = (max(row - 1, 0), col)
                if spt[next_r][next_c][1] >= cur_dist + 1 and desc[next_r][next_c]!= 3:
                    if spt[next_r][next_c][1] > cur_dist + 1:   
                        spt[next_r][next_c][1] = cur_dist + 1
                        q.put([(next_r, next_c), cur_dist + 1])
                    spt[next_r][next_c][0].append(S)
                        
            elif action == 'E':
                (next_r, next_c) = (row, min(col + 1, max_col))
                
                if spt[next_r][next_c][1] >= cur_dist + 1 and desc[next_r][next_c]!= 3:
                    if spt[next_r][next_c][1] > cur_dist + 1:   
                        spt[next_r][next_c][1] = cur_dist + 1
                        q.put([(next_r, next_c), cur_dist + 1])
                    spt[next_r][next_c][0].append(W)
            
            elif action == 'W':
                (next_r, next_c) = (row, max(col - 1, 0))
                
                if spt[next_r][next_c][1] >= cur_dist + 1 and desc[next_r][next_c]!= 3:
                    if spt[next_r][next_c][1] > cur_dist + 1:
                        spt[next_r][next_c][1] = cur_dist + 1
                        q.put([(next_r, next_c), cur_dist + 1])
                    spt[next_r][next_c][0].append(E)

    return spt

In [21]:
TOY_MAP = np.array([
    [2,0,0,0,0],
    [0,0,0,0,0],
    [0,3,0,3,0],
    [0,0,0,3,0],
    [0,3,0,0,9]
    ])

In [22]:
# for the i,jth entry, return a list of equally optimal directions to move in 
# and a number representing steps needed to reach goal (hive)
# 'inf' refers to squares from which hive unreachable; e.g. inside obstacle locations
expert_navigation_policy_set(TOY_MAP, (0,0))

[[[[-1], 0], [['W'], 1], [['W'], 2], [['W'], 3], [['W'], 4]],
 [[['N'], 1],
  [['N', 'W'], 2],
  [['N', 'W'], 3],
  [['N', 'W'], 4],
  [['N', 'W'], 5]],
 [[['N'], 2], [[], inf], [['N'], 4], [[], inf], [['N'], 6]],
 [[['N'], 3], [['W'], 4], [['N', 'W'], 5], [[], inf], [['N'], 7]],
 [[['N'], 4], [[], inf], [['N'], 6], [['W'], 7], [['N', 'W'], 8]]]

In [23]:
# pull just the optimal directions for readability
[[q[0] for q in r] for r in expert_navigation_policy_set(TOY_MAP, (0,0))]

[[[-1], ['W'], ['W'], ['W'], ['W']],
 [['N'], ['N', 'W'], ['N', 'W'], ['N', 'W'], ['N', 'W']],
 [['N'], [], ['N'], [], ['N']],
 [['N'], ['W'], ['N', 'W'], [], ['N']],
 [['N'], [], ['N'], ['W'], ['N', 'W']]]

In [34]:
# example of getting list of optimal directions for square i,j
i = 4
j = 4
expert_navigation_policy_set(TOY_MAP, (0,0))[i][j][0]

['N', 'W']

In [ ]:
make & store shortest path tree back to hive

agent_next_action():
    if agent carrying food:
        follow shortest path back to hive, possibly with noise
        
    else if agent not carrying food:
        get partial observability window
        look at pheromone levels of cells (num varies depending on obs_rad) in continued direction of last step
        move in direction of cell with greatest pheromone
        if that cell is diagonal and both not blocked, pick direction to take randomly
        if all those cells are blocked, randomly choose an unblocked direction to take
        return/store action taken
    
    
agent.get_action(partially-observable observation): returns (movement, pheromone)
        

In [2]:
# q: can ants see pheromones through walls? (going to assume yes for simplicity)
def get_action(self, obs, valid_actions):
    pos_in_window = obs_rad, obs_rad
    if self.food:
        # take optimal direction towards hive
        return np.random.choice(spt[self.x][self.y][0])
    else:
        if last_action == 'N':
            best_loc = np.argmax(obs[:obs_rad, :])
            
        elif last_action == 'E':
            best_loc = np.argmax(obs[:, obs_rad + 1:])
        
        elif last_action = 'S':
            best_loc = np.argmax(obs[obs_rad + 1:, :])
        
        elif last_action = 'W':
            best_loc =  np.argmax(obs[:, :obs_rad])
            
        else: 
            assert False
            
    good_actions = []     
    # if best action row > agent row in observation window
    if best_loc[0] > obs_rad:
        good_actions.append(S)
    if best_loc[0] < obs_rad:
        good_actions.append(N)
    if best_loc[1] > obs_rad:
        good_actions.append(E)
    if best_loc[1] < obs_rad:
        good_actions.append(W)
        
    # if list empty take random action from all actions not blocked by obstacles
    good_actions = np.intersect1d(np.array(good_answers), valid_actions)
    if len(good_actions) > 0:
        return np.random.choice(good_actions)
    else:
        return np.random.choice(valid_actions)
    
    
        

SyntaxError: invalid syntax (<ipython-input-2-b75baaba8503>, line 14)

In [32]:
z = np.array([[0,1,2,3,4,],[5,6,7,8,9]])
z

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [33]:
z[:,2:]

array([[2, 3, 4],
       [7, 8, 9]])